In [1]:
#include "xeus/xcomm.hpp"

# Target

In [2]:
namespace xeus
{
    static inline void xwidget_comm_opened(const xcomm& comm, const xeus::xmessage& msg)
    {
        std::cout << "Hello World" << std::endl;
    }
}

In [3]:
auto& interpreter = ::xeus::get_interpreter();
interpreter.comm_manager().register_comm_target("jupyter.widget", xeus::xwidget_comm_opened);

auto& manager = interpreter.comm_manager();
auto target = manager.target("jupyter.widget");

# Models

In [4]:
xeus::xcomm layout_comm(target, xeus::xguid());
xeus::xjson layout_data = R"(
{
    "_model_module": "jupyter-js-widgets",
    "_model_module_version": "~2.1.4",
    "_model_name": "LayoutModel",
    "_view_module": "jupyter-js-widgets",
    "_view_module_version": "~2.1.4",
    "_view_name": "LayoutView"
}
)"_json;

layout_comm.open(xeus::xjson::object(), std::move(layout_data))

In [5]:
xeus::xcomm slider_comm(target, xeus::xguid());
xeus::xjson slider_data = R"(
{
    "_model_module": "jupyter-js-widgets",
    "_model_module_version": "~2.1.4",
    "_model_name": "IntSliderModel",
    "_view_module": "jupyter-js-widgets",
    "_view_module_version": "~2.1.4",
    "_view_name": "IntSliderView",
    "value": "50"
}
)"_json;

slider_data["layout"] = "IPY_MODEL_" + xeus::guid_to_hex(layout_comm.id());
slider_comm.open(xeus::xjson::object(), std::move(slider_data))

# Display

In [6]:
xeus::xjson display_data = R"(
{
    "application/vnd.jupyter.widget-view+json": {
        "version_major": "2",
        "version_minor": "0"
    }
}
)"_json;

display_data["application/vnd.jupyter.widget-view+json"]["model_id"] = xeus::guid_to_hex(slider_comm.id());

interpreter.display_data(
    std::move(display_data),
    xeus::xjson::object(),
    xeus::xjson::object()
)

In [7]:
xeus::xjson comm_display = R"(
{
    "data": {
        "method": "display"
    }
}
)"_json;

comm_display["comm_id"] =  xeus::guid_to_hex(slider_comm.id());

target->publish_message("comm_msg", xeus::xjson::object(), std::move(comm_display));